# **Klasifikasi DC-08 Ai Learning Insight**

## Deskripsi

isi juga

## **Informasi Dataset**

dataset ini menyajikan hasil dari **isi yak**

## Fitur Utama
fungsi kolom...

## **1. Import Library**

In [19]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import joblib


## **2. Load Dataset**

In [20]:
df = pd.read_csv("df_classification.csv")
df.head()


,user_id,avg_time_to_complete,total_completed_modules,active_days,repeat_ratio,avg_submission_rating,avg_exam_score,exam_pass_rate,learner_type,learner_label
0,3390,0.226649,11614,230,0.817295,1.620495,79.915323,0.788978,Consistent Learner,0
1,5774,1.583095,2061,94,0.172611,1.723022,70.225397,0.609524,Consistent Learner,0
2,11836,1.392633,2868,104,0.594613,1.681967,85.585965,0.915789,Consistent Learner,0
3,17833,0.513218,7484,198,0.824997,3.023378,83.822018,0.856881,Consistent Learner,0
4,32258,2.332081,8028,235,0.805614,2.538160,84.257925,0.841499,Consistent Learner,0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   user_id                  31 non-null     int64  
 1   avg_time_to_complete     31 non-null     float64
 2   total_completed_modules  31 non-null     int64  
 3   active_days              31 non-null     int64  
 4   repeat_ratio             31 non-null     float64
 5   avg_submission_rating    31 non-null     float64
 6   avg_exam_score           31 non-null     float64
 7   exam_pass_rate           31 non-null     float64
 8   learner_type             31 non-null     object 
 9   learner_label            31 non-null     int64  
dtypes: float64(5), int64(4), object(1)
memory usage: 2.6+ KB


## **3. Featurin Column**

In [22]:
feature_cols = [
    "avg_time_to_complete",
    "total_completed_modules",
    "active_days",
    "repeat_ratio",
    "avg_submission_rating",
    "avg_exam_score",
    "exam_pass_rate"
]

X = df[feature_cols]
y = df["learner_label"]

## **4. Split Dataset**

In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

## **5. Scaling**

In [24]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Simpan scaler untuk web nanti
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

## **6. Training**

### Algoritma Klasik

Logistic Regresion

In [25]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_scaled, y_train)

y_pred_lr = lr.predict(X_test_scaled)
acc_lr = accuracy_score(y_test, y_pred_lr)

print("Logistic Regression Accuracy:", acc_lr)

Logistic Regression Accuracy: 1.0


Random FOrest

In [26]:
rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)
rf.fit(X_train, y_train)  # RF tidak wajib scaling

y_pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)

print("Random Forest Accuracy:", acc_rf)

Random Forest Accuracy: 0.8571428571428571


SVM

In [27]:
svm = SVC(kernel="rbf")
svm.fit(X_train_scaled, y_train)

y_pred_svm = svm.predict(X_test_scaled)
acc_svm = accuracy_score(y_test, y_pred_svm)

print("SVM Accuracy:", acc_svm)

SVM Accuracy: 0.8571428571428571


### Algoritma Dep LEarning

In [28]:
pip install tensorflow

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

num_classes = y.nunique()

model_dl = Sequential([
    Dense(32, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    Dense(16, activation="relu"),
    Dense(num_classes, activation="softmax")
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
model_dl.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [31]:
model_dl.fit(
    X_train_scaled, y_train,
    epochs=50,
    batch_size=8,
    validation_split=0.2,
    verbose=0
)

In [32]:
loss, acc_dl = model_dl.evaluate(X_test_scaled, y_test, verbose=0)
print("Deep Learning Accuracy:", acc_dl)

Deep Learning Accuracy: 0.8571428656578064


### **7. Evaluasi Model**

In [33]:
result = pd.DataFrame({
    "Model": ["Logistic Regression", "Random Forest", "SVM", "Deep Learning"],
    "Accuracy": [acc_lr, acc_rf, acc_svm, acc_dl]
})

print(result)


                 Model  Accuracy
0  Logistic Regression  1.000000
1        Random Forest  0.857143
2                  SVM  0.857143
3        Deep Learning  0.857143


### **8. SImpan Model**

In [34]:
import joblib
joblib.dump(rf, "model_learner_classifier.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [35]:
import joblib
joblib.dump(rf, "model_learner_classifier.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

## **inference**

In [41]:
import pandas as pd
import joblib
import numpy as np

scaler = joblib.load("scaler.pkl")
model = joblib.load("model_learner_classifier.pkl")

feature_cols = [
    "avg_time_to_complete",
    "total_completed_modules",
    "active_days",
    "repeat_ratio",
    "avg_submission_rating",
    "avg_exam_score",
    "exam_pass_rate"
]

new_user_df = pd.DataFrame([{
    "avg_time_to_complete": 0.8,
    "total_completed_modules": 5000,
    "active_days": 120,
    "repeat_ratio": 0.7,
    "avg_submission_rating": 2.5,
    "avg_exam_score": 82,
    "exam_pass_rate": 0.85,
}])

new_user_scaled = scaler.transform(new_user_df[feature_cols])
pred_label = model.predict(new_user_scaled)[0]

label_to_class = {
    0: "Consistent Learner",
    1: "Fast Learner",
    2: "Reflective Learner"
}

print("Predicted label:", pred_label)
print("Predicted learner type:", label_to_class[pred_label])

Predicted label: 2
Predicted learner type: Reflective Learner


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
